### Important Note
I have took out the comments we have made during the class all comments/markdowns have specifically been made to tell what I'm changing and why.

In [22]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

In [23]:
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="rgb_array")

grid_height, grid_width = env.unwrapped.desc.shape

env.reset()

(0, {'prob': 1})

In [24]:
gamma = 0.9

max_epsilon = 0.9
epsilon = max_epsilon
min_epsilon = 0.01

decay = 0.0005

learning_rate = 0.5

trainings = 50
total_episodes = 2_500
max_steps = 50
print(f'Total steps: {trainings * total_episodes * max_steps}')
print(f'Estimated time: {total_episodes * trainings / 10_000 * 6 / 60} minutes')  # just based on my laptop's performance

Total steps: 6250000
Estimated time: 1.25 minutes


In [25]:
def calculate_value(Q, state, action, reward, next_state, alpha, gamma):
    Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
    return Q

In [26]:
# helper for Q-values visualisation
def summarise_best(q_table):  # removed width and height from incoming variables as the function can already access that
    best_values = np.max(q_table, axis = 1)
    return best_values.reshape(grid_width, grid_height)

In [27]:
first_solves = []
step_counts = []

for training in range(trainings):
    first_solve = (-1, -1)
    min_step_count = (-1, max_steps + 1) # episode, step count
    q_table = np.zeros([env.observation_space.n, env.action_space.n])

    for episode in range(total_episodes):
        state, info = env.reset()

        done = False
        total_rewards = 0

        for step in range(max_steps):
            # next action
            if np.random.uniform(0, 1) > epsilon:
                action = np.argmax(q_table[state, :])
            else:
                action = env.action_space.sample()

            # results
            observation, reward, done, truncated, info = env.step(action)

            # q_table update
            q_table = calculate_value(q_table, state, action, reward, observation, learning_rate, gamma)

            # update: state, reward
            state = observation

            if reward == 1:
                if first_solve == (-1, -1):
                    first_solve = (episode, step)
                if min_step_count[1] > step:
                    min_step_count = (episode, step)

            total_rewards += reward - 1

            # end condition check
            if done:
                #print(f"Step count: {step}")
                break

        # decay
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
        #print(f"Episode {episode} done with current reward: {total_rewards}\nEpsilon: {epsilon}")

    first_solves.append(first_solve)
    step_counts.append(min_step_count)

In [28]:
# average out the results
average_frist_solve = [0, 0]
average_step_count = [0, 0]
for i in range(trainings):
    average_frist_solve[0] += first_solves[i][0]
    average_frist_solve[1] += first_solves[i][1]
    average_step_count[0] += step_counts[i][0]
    average_step_count[1] += step_counts[i][1]

average_frist_solve[0] = average_frist_solve[0] / trainings
average_frist_solve[1] = average_frist_solve[1] / trainings
average_step_count[0] = average_step_count[0] / trainings
average_step_count[1] = average_step_count[1] / trainings

print(f'First average solve episode: {average_frist_solve[0]}, with average step count: {first_solve[1]}')
print(f'Smallest average step count of {average_step_count[1]} reached on average episode {average_step_count[0]}')

First average solve episode: 102.3, with average step count: 12
Smallest average step count of 5.92 reached on average episode 291.52


with -0.01 per step it made an about 10% decrease in needed episodes for both values compared to the "basic_version"

same outcome with -0.1 per step

interestingly with a fairly wild -1 reward per step it cut first solve episode by 25% but the optimization only by 15%ish